In [ ]:
import numpy as np
import os

In [ ]:
data_dir = 'sonic_pi_face/data/'
filename = 'data-20-43-23-5-2017.txt'
with open(os.path.join(data_dir,filename), 'r') as file:
    raw = file.read()

In [ ]:
frames = raw.split('\n')

In [ ]:
data_list = []
for frame in frames:
    data_list.append(frame.split(','))

In [ ]:
data = np.asarray(data_list[:][:-1])

In [ ]:
# Remove trailing whitespace
data = np.delete(data,-1,-1)

In [ ]:
data = data.reshape(10,40,40)

In [ ]:
dcopy = data.copy()

In [ ]:
dcopy = dcopy[...,np.newaxis]

In [ ]:
ccopy = dcopy.copy()
def split(a):
#     return a
    return tuple((float(a[0][:9]),float(a[0][9:])))
splitData =np.apply_along_axis(split,3,ccopy)
x = splitData[...,::2]
y = splitData[...,1::2]